## Preliminaries

In [2]:
import cv2
import numpy as np
import pandas as pd

## Point coordinate detection

In [5]:
# Load image from Pictures/ directory
img = cv2.imread('Pictures/3.png')

# Convert img to HSV format
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV threshold range for neon green points
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])

# Apply threshold on the image
mask = cv2.inRange(hsv, lower_green, upper_green)

# Clean up mask, merge broken regions and remove noise 
kernel = np.ones((5,5), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Connected components
num, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

points = []
for i in range(1, num):  # skip background (label 0)
    cx, cy = centroids[i]
    points.append((int(cx), int(cy)))

# Convert to DataFrame
df = pd.DataFrame(points, columns=["x", "y"])

print(df.head())

      x    y
0   253  203
1  1975  217
2  1191  626
3   627  723
4  1670  732


## Draw red dots on given green dots for verification

In [4]:
# Load image
img = cv2.imread('Pictures/3.png')
output = img.copy()

# Convert to HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Define HSV range for neon green (adjust if needed)
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])

# Threshold the image
mask = cv2.inRange(hsv, lower_green, upper_green)

# Find contours
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Appending and drawing points
points = []
for cnt in contours:
    M = cv2.moments(cnt)
    if M["m00"] > 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        points.append((cx, cy))
        
        # Draw a red circle on each detected point
        cv2.circle(output, (cx, cy), 10, (0, 0, 255), -1)  # red filled circle

# Convert to DataFrame
df = pd.DataFrame(points, columns=["x", "y"])
print(df)

# Save output
cv2.imwrite("detected_points.png", output)

       x     y
0    253  2671
1   1993  2642
2   1279  2618
3   1489  2579
4    965  2563
5   1672  2241
6    634  2241
7   1496  2231
8    976  2229
9   1489  2048
10   980  2047
11  1287  2045
12  1191  2002
13  1679  1988
14   639  1986
15  1197  1796
16  1672  1741
17   636  1736
18  1190  1579
19    54  1573
20  1672  1489
21   634  1482
22  1194  1365
23  1670  1239
24   632  1230
25  1191  1149
26  1672   984
27   627   975
28  1195   933
29  1670   732
30   627   723
31  1191   626
32  1975   217
33   253   203


True